In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())
%cd ../
import pylib.mix as mix
%cd "./jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP/"

25244
/media/work/docs/codes/QuCF/scripts-py
/media/work/docs/codes/QuCF/scripts-py/jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP


In [2]:
mix.reload_module(mix)

# -----------------------------------------------------------
def find_exp(number) -> int:
    base10 = np.log10(abs(number))

    temp = np.floor(base10)
    flag_neg = False
    if temp < 0:
        flag_neg = True

    return abs(temp), flag_neg
# -----------------------------------------------------------
def txt_to_hdf5(old_filename, new_filename):
    function_type_res = "QSP-ham"
    rescaling_factor_res = 1.
    parity_res = -1

    # --- read the .angles (text) file ---
    print("read angles from:\n " + old_filename)
    with open(old_filename) as f:
        param_res = float(f.readline()[:-1]) 
        eps_res   = float(f.readline()[:-1]) 
        Na        = int(f.readline()[:-1]) 
        angles = np.zeros(Na)
        for ii in range(Na):
            angles[ii] = float(f.readline())
            
    print()
    print("function type: \t\t{:s}".format(function_type_res))    
    print("rescaling factor: \t{:0.3f}".format(rescaling_factor_res))
    print("parity: \t\t{:d}".format(parity_res))
    print("parameter: \t\t{:0.3e}".format(param_res))
    print("error: \t\t\t{:0.3e}".format(eps_res))
    print("N-angles: \t\t{:d}".format(Na))
    print()       

    # --- write the .hdf5 file ---
    print("write angles to:\n " + new_filename)
    with h5py.File(new_filename, "w") as f:
        grp = f.create_group("basic")
        
        grp.create_dataset('polynomial_type',  data=function_type_res)
        grp.create_dataset('rescaling_factor', data=float(rescaling_factor_res))
        grp.create_dataset('parity',           data=parity_res)
        grp.create_dataset('eps',              data=eps_res)
        grp.create_dataset('par',              data=param_res)

        grp = f.create_group("angles")
        grp.create_dataset('QSP-ham',  data = angles)
    return

In [7]:
# ----------------------------------------------------------
# --- Specify QSP parameters ---
# ----------------------------------------------------------
t_moments = [3.904120190901e+02]

Nt = len(t_moments)
eps_QSP_array = [1e-12] * Nt

case_format = 1

# --- Form the name of the output name ---
file_names = [None] * Nt
print("--- Chosen QSP parameters ---")
if case_format == 1:
    for it in range(Nt):
        eps_QSP = eps_QSP_array[it]
        t1 = t_moments[it]
        exp_t1, flag_neg = find_exp(t1)

        str_exp = "p"
        if flag_neg:
            str_exp = "m"

        if flag_neg:
            temp_int = int(t1 * 10**(exp_t1+3))
        else:
            temp_int = int(t1 / 10**(exp_t1-3))

        file_names[it] = "QSP_t{:d}{:s}{:d}_eps{:d}".format(
            temp_int, 
            str_exp,
            int(exp_t1),
            int(abs(np.round(np.log10(eps_QSP))))
        )

        print("t, eps: {:0.3e}, {:0.3e}".format(t1, eps_QSP))
        print("file name: {:s}".format(file_names[it]))
        print()

--- Chosen QSP parameters ---
t, eps: 3.904e+02, 1.000e-12
file name: QSP_t3904p2_eps12



In [8]:
# ------------------------------------------------
# --- Compute QSP angles ---
# ------------------------------------------------
line_code = "/media/work/docs/codes/progs/Q#F#/test-qsp/bin/Debug/netcoreapp3.1/qsp"
for it in range(Nt):
    eps_QSP = eps_QSP_array[it]
    t1 = t_moments[it]

    line_cmd = "{:s} {:0.3e} {:0.3e} {:s} {:s}".format(
        line_code, t1, eps_QSP, "./", file_names[it]
    )
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

    # from the .txt file to .hdf5 file:
    txt_to_hdf5(file_names[it], file_names[it] + ".hdf5")

    # remove the .txt file:
    line_cmd = "rm {:s}".format(file_names[it])
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

Start calculating...
Chosen tau = 390.4
Chosen eps = 1E-12
polyA = { Denominator = 4503599627370496
  Numerator =
             1z^-468 + 3z^-466 + 12z^-464 + 41z^-462 + 135z^-460 + 439z^-458 + 1401z^-456 + 4398z^-454 + 13567z^-452 + 41133z^-450 + 122526z^-448 + 358498z^-446 + 1029964z^-444 + 2904601z^-442 + 8037533z^-440 + 21815522z^-438 + 58055056z^-436 + 151412177z^-434 + 386839799z^-432 + 967702234z^-430 + 2369022644z^-428 + 5672500161z^-426 + 13277033739z^-424 + 30357949664z^-422 + 67762558538z^-420 + 147546414147z^-418 + 313137521282z^-416 + 647174110130z^-414 + 1301244503260z^-412 + 2542574280171z^-410 + 4822082362979z^-408 + 8864282468809z^-406 + 15769564974353z^-404 + 27100639626479z^-402 + 44896209293272z^-400 + 71520183658404z^-398 + 109227450675249z^-396 + 159334293500802z^-394 + 220961728739633z^-392 + 289510774492776z^-390 + 355335849677227z^-388 + 403436159268603z^-386 + 415200384227378z^-384 + 373026726276925z^-382 + 267710637367102z^-380 + 106794974122326z^-378 + -79909